# Description 
This script scrapes the stoiximan webpage. It provides a dataframe of all the upcoming EPL matches that stoiximan provides together with the odds of the Betting company. It then joins them with the 'GatheredData/eplGMTStdTimeFiles/epl-2023-GMTStandardTime.csv' file and makes a 'Newfixtures.csv' file. In the end it runs all the needed scripts so that the predictor runs correctly for each round.

In [1]:
import requests 
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import datetime

url = 'https://en.stoiximan.gr/sport/soccer/england/premier-league/1/'
page = requests.get(url)

soup = BeautifulSoup(page.content, "html.parser")

In [2]:
# A function to change team names as depicted in Stoiximan in the way they are depicted in the files
def replace_names(string):
    old_names = ['Fulham FC', 'Everton FC', 'Leeds United', 'Southampton FC', 'West Ham United', 'Arsenal FC', 'AFC Bournemouth', 'Crystal Palace FC', 'Liverpool FC', 'Brighton & Hove Albion', 'Chelsea FC', 'Tottenham Hotspur', 'Brentford FC', 'Burnley FC', 'Luton Town']
    new_names = ['Fulham', 'Everton', 'Leeds', 'Southampton', 'West Ham', 'Arsenal', 'Bournemouth', 'Crystal Palace', 'Liverpool', 'Brighton', 'Chelsea', 'Tottenham', 'Brentford', 'Burnley', 'Luton']
    
    for i in range(len(old_names)):
        string = string.replace(old_names[i], new_names[i])
    return string

In [3]:

soup = soup.body
script = soup.find_all("script")
script = script[0]

print(script)

<script>window["initial_state"]={"data":{"leagueStandings":[],"leagueGrouping":true,"sport":{"id":"FOOT","name":"Soccer","region":"England","url":"/sport/soccer/"},"parentMarkets":[{"id":"0","name":"Matches","selected":true,"markets":[{"id":"0","name":"Main","selected":true,"type":"Main"},{"id":"1","name":"Double chance"},{"id":"2","name":"Half Time"},{"id":"3","name":"Asian Lines"},{"id":"4","name":"Corner kicks"},{"id":"5","name":"Players","type":"Players"}]},{"id":"6","name":"Long term bets","type":"LongtermBets"}],"markets":[],"blocks":[{"name":"England - Premier League","shortName":"Premier League","id":"1r","url":"/sport/soccer/england/premier-league/1/","headers":[{"marketTypes":["MR12","MRES"],"columns":[{"columnSpan":1,"name":"1"},{"columnSpan":1,"name":"X"},{"columnSpan":1,"name":"2"}]},{"marketTypes":["HCTG"],"columns":[{"columnSpan":1,"name":"OVER/UNDER"}]},{"marketTypes":["BTSC"],"columns":[{"columnSpan":1,"name":"GG/NG"}]}],"layout":2,"events":[{"stats":[{"url":"https://s

In [4]:
Odds = []
Dates = []
HomeTeams = []
AwayTeams = []

print(script)
    
# Define the regular expression pattern
pattern = r'<script>(.*?)<\/script>'

# Use re.findall to find all matches
matches = re.findall(pattern, str(script), re.DOTALL)

if matches:
    # Extract the content between script tags
    content_between_scripts = matches[0]
    content_between_scripts= content_between_scripts.split('window["initial_state"]=')[1].strip()
    #print(content_between_scripts)
else:
    print('No match found.')
    
d = json.loads(content_between_scripts)


for match in d['data']['blocks'][0]['events']:
    teams = match['shortName'].split('-')
    HomeTeams.append(replace_names(teams[0]))
    AwayTeams.append(replace_names(teams[1]))
    Dates.append(datetime.datetime.fromtimestamp(match['startTime']/1000.0).strftime('%d/%m/%Y'))
    for x in match['markets']:
        for y in x['selections']:
            Odds.append(y['price'])
            


rows = []
while Odds:
    rows.append(Odds[:7])
    Odds = Odds[7:]

print(rows)
print(Dates)

<script>window["initial_state"]={"data":{"leagueStandings":[],"leagueGrouping":true,"sport":{"id":"FOOT","name":"Soccer","region":"England","url":"/sport/soccer/"},"parentMarkets":[{"id":"0","name":"Matches","selected":true,"markets":[{"id":"0","name":"Main","selected":true,"type":"Main"},{"id":"1","name":"Double chance"},{"id":"2","name":"Half Time"},{"id":"3","name":"Asian Lines"},{"id":"4","name":"Corner kicks"},{"id":"5","name":"Players","type":"Players"}]},{"id":"6","name":"Long term bets","type":"LongtermBets"}],"markets":[],"blocks":[{"name":"England - Premier League","shortName":"Premier League","id":"1r","url":"/sport/soccer/england/premier-league/1/","headers":[{"marketTypes":["MR12","MRES"],"columns":[{"columnSpan":1,"name":"1"},{"columnSpan":1,"name":"X"},{"columnSpan":1,"name":"2"}]},{"marketTypes":["HCTG"],"columns":[{"columnSpan":1,"name":"OVER/UNDER"}]},{"marketTypes":["BTSC"],"columns":[{"columnSpan":1,"name":"GG/NG"}]}],"layout":2,"events":[{"stats":[{"url":"https://s

In [5]:
df_teams = pd.DataFrame({'Home Team': HomeTeams, 'Away Team': AwayTeams})
df_values = pd.DataFrame(rows, columns=['1', 'x', '2', 'Over', 'Under', 'Yes', 'No'])
df_all = pd.concat([df_teams, df_values], axis=1)
df_all['Home Team'] = df_all['Home Team'].str.strip()
df_all['Away Team'] = df_all['Away Team'].str.strip()
display(df_all)

,Home Team,Away Team,1,x,2,Over,Under,Yes,No
0,Manchester City,Nottingham Forest,1.17,7.50,16.00,2.02,1.80,2.15,1.65
1,Luton,Wolverhampton Wanderers,3.30,3.55,2.15,1.91,1.90,1.75,2.02
2,Crystal Palace,Fulham,1.91,3.60,4.00,2.00,1.82,1.85,1.90
3,Brentford,Everton,1.80,3.75,4.40,1.90,1.91,1.80,1.93
4,Burnley,Manchester United,4.40,4.00,1.75,1.62,2.30,1.60,2.25
5,Chelsea,Aston Villa,1.90,3.80,3.85,1.70,2.18,1.60,2.25
6,Liverpool,West Ham,1.45,4.90,6.50,2.05,1.78,1.57,2.32
7,Brighton,Bournemouth,1.38,5.10,7.70,1.93,1.88,1.55,2.37
8,Arsenal,Tottenham,1.72,4.20,4.25,2.18,1.70,1.50,2.47
9,Sheffield United,Newcastle United,6.60,4.55,1.47,1.65,2.27,1.78,1.95


In [6]:
df = pd.read_csv('GatheredData/eplGMTStdTimeFiles/epl-2023-GMTStandardTime.csv')

display(df)

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,1,1,11/08/2023 20:00,Turf Moor,Burnley,Man City,NaN
1,2,1,12/08/2023 12:30,Emirates Stadium,Arsenal,Nottingham Forest,NaN
2,3,1,12/08/2023 15:00,Vitality Stadium,Bournemouth,West Ham,NaN
3,4,1,12/08/2023 15:00,Amex Stadium,Brighton,Luton,NaN
4,5,1,12/08/2023 15:00,Goodison Park,Everton,Fulham,NaN
...,...,...,...,...,...,...,...
375,376,38,19/05/2024 16:00,Selhurst Park,Crystal Palace,Aston Villa,NaN
376,377,38,19/05/2024 16:00,Anfield,Liverpool,Wolves,NaN
377,378,38,19/05/2024 16:00,Kenilworth Road,Luton,Fulham,NaN
378,379,38,19/05/2024 16:00,Etihad Stadium,Man City,West Ham,NaN


In [7]:
df_merged = pd.merge(df, df_all, on=['Home Team', 'Away Team'], how='right')
df_merged['Date']=Dates
display(df_merged)
df_merged.to_csv('UpcomingFixtures/Newfixtures.csv')

,Match Number,Round Number,Date,Location,Home Team,Away Team,Result,1,x,2,Over,Under,Yes,No
0,NaN,NaN,23/09/2023,NaN,Manchester City,Nottingham Forest,NaN,1.17,7.50,16.00,2.02,1.80,2.15,1.65
1,NaN,NaN,23/09/2023,NaN,Luton,Wolverhampton Wanderers,NaN,3.30,3.55,2.15,1.91,1.90,1.75,2.02
2,56.0,6.0,23/09/2023,Selhurst Park,Crystal Palace,Fulham,NaN,1.91,3.60,4.00,2.00,1.82,1.85,1.90
3,52.0,6.0,23/09/2023,Gtech Community Stadium,Brentford,Everton,NaN,1.80,3.75,4.40,1.90,1.91,1.80,1.93
4,NaN,NaN,23/09/2023,NaN,Burnley,Manchester United,NaN,4.40,4.00,1.75,1.62,2.30,1.60,2.25
5,55.0,6.0,24/09/2023,Stamford Bridge,Chelsea,Aston Villa,NaN,1.90,3.80,3.85,1.70,2.18,1.60,2.25
6,57.0,6.0,24/09/2023,Anfield,Liverpool,West Ham,NaN,1.45,4.90,6.50,2.05,1.78,1.57,2.32
7,53.0,6.0,24/09/2023,Amex Stadium,Brighton,Bournemouth,NaN,1.38,5.10,7.70,1.93,1.88,1.55,2.37
8,NaN,NaN,24/09/2023,NaN,Arsenal,Tottenham,NaN,1.72,4.20,4.25,2.18,1.70,1.50,2.47
9,NaN,NaN,24/09/2023,NaN,Sheffield United,Newcastle United,NaN,6.60,4.55,1.47,1.65,2.27,1.78,1.95


In [8]:
#Downloading the updated results
import os
import requests

# URL of the file to download
url = "https://www.football-data.co.uk/mmz4281/2324/E0.csv"

# Name for the downloaded file
file_name = "E0-2023.csv"

# Path to the directory where you want to save the file
output_directory = "GatheredData/ExFiles"

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Full path for the downloaded file
output_path = os.path.join(output_directory, file_name)

# Download the file
response = requests.get(url)
if response.status_code == 200:
    with open(output_path, "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved as {output_path}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

File downloaded and saved as GatheredData/ExFiles/E0-2023.csv


In [9]:
#Changing team names
temp_pd = pd.read_csv('GatheredData/ExFiles/E0-2023.csv')
team_dict = {
                     'Man City': 'Manchester City',
                     'Man United': 'Manchester United',
                     'Nott\'m Forest': 'Nottingham Forest',
                     'Newcastle': 'Newcastle United',
                     'Wolves': 'Wolverhampton Wanderers'
                         }
temp_pd['HomeTeam'] = temp_pd['HomeTeam'].replace(team_dict)
temp_pd['AwayTeam'] = temp_pd['AwayTeam'].replace(team_dict)

temp_pd.to_csv('GatheredData/ExFiles/E0-2023.csv')

In [10]:
# run the script named "my_script.py" with argument "arg1"
os.system("python UpcomingFixtures/NewFixturesScraping2.py")
os.system("python UpcomingFixtures/NewfixturesScraping2Month.py")
os.system("python UpcomingFixtures/NewfixturesScraping2Month3.py")
os.system("python GatheringScripts/Gather2023Season.py")
os.system("python GatheringScripts/Gather2023SeasonForm.py")
os.system("python GatheringScripts/Gather2023SeasonForm_3Month.py")

Traceback (most recent call last):
  File "/home/nickpad/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewFixturesScraping2.py", line 127, in <module>
    loop.run_until_complete(main())
  File "/usr/lib64/python3.11/asyncio/base_events.py", line 653, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/home/nickpad/Documents/EnglishPremierLeaguePredictor/UpcomingFixtures/NewFixturesScraping2.py", line 15, in main
    df = pd.read_csv('Newfixtures.csv')
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nickpad/.local/lib/python3.11/site-packages/pandas/util/_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/nickpad/.local/lib/python3.11/site-packages/pandas/util/_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/home/nickpad/.local/lib/python3.11/site-packages/pandas/io/parsers/readers.py", line 950, in read_csv


11/08/2023
2023-08-10
2023-08-10
0    Burnley
Name: HomeTeam, dtype: object
0    Manchester City
Name: AwayTeam, dtype: object
0      Team  M  G GA PTS xG xGA PPDA OPPDA  DC  ODC  PTS/M
13  Burnley  0  0  0   0  0   0    0     0   0    0      0
0              Team  M  G GA PTS xG xGA PPDA OPPDA  DC  ODC  PTS/M
11  Manchester City  0  0  0   0  0   0    0     0   0    0      0
12/08/2023
2023-08-11
2023-08-11
1             Arsenal
2         Bournemouth
3            Brighton
4             Everton
5    Sheffield United
6    Newcastle United
Name: HomeTeam, dtype: object
1    Nottingham Forest
2             West Ham
3                Luton
4               Fulham
5       Crystal Palace
6          Aston Villa
Name: AwayTeam, dtype: object


/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023Season.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023Season.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023Season.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy 

0               Team  M  G GA PTS xG xGA PPDA OPPDA  DC  ODC  PTS/M
9            Arsenal  0  0  0   0  0   0    0     0   0    0      0
4        Bournemouth  0  0  0   0  0   0    0     0   0    0      0
13          Brighton  0  0  0   0  0   0    0     0   0    0      0
3            Everton  0  0  0   0  0   0    0     0   0    0      0
16  Sheffield United  0  0  0   0  0   0    0     0   0    0      0
10  Newcastle United  0  0  0   0  0   0    0     0   0    0      0
0                Team  M  G GA PTS xG xGA PPDA OPPDA  DC  ODC  PTS/M
18  Nottingham Forest  0  0  0   0  0   0    0     0   0    0      0
7            West Ham  0  0  0   0  0   0    0     0   0    0      0
19              Luton  0  0  0   0  0   0    0     0   0    0      0
14             Fulham  0  0  0   0  0   0    0     0   0    0      0
5      Crystal Palace  0  0  0   0  0   0    0     0   0    0      0
2         Aston Villa  0  0  0   0  0   0    0     0   0    0      0
13/08/2023
2023-08-12
2023-08-12
7    Bre

/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023Season.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023Season.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


...

/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm_3Month.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm_3Month.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


18/09/2023
2023-09-17
2023-09-17
48    Nottingham Forest
Name: HomeTeam, dtype: object
48    Burnley
Name: AwayTeam, dtype: object
0                Team  M  G GA PTS    xG   xGA   PPDA OPPDA    DC   ODC PTS/M
12  Nottingham Forest  4  6  6   6  4.45  6.71  22.93  7.43  4.25  11.0   1.5


/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm_3Month.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HomeStats['Team'] = pd.Categorical(
/home/nickpad/Documents/EnglishPremierLeaguePredictor/GatheringScripts/Gather2023SeasonForm_3Month.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AwayStats['Team'] = pd.Categorical(


0      Team  M  G  GA PTS    xG   xGA   PPDA  OPPDA        DC        ODC PTS/M
19  Burnley  3  3  11   0  2.61  7.64  12.47  16.79  6.333333  11.333333   0.0
                      HTeam  HM  HG  ...        HDC       HODC    HPTS/M
0                   Burnley   0   0  ...   0.000000   0.000000  0.000000
1                   Arsenal   0   0  ...   0.000000   0.000000  0.000000
2               Bournemouth   0   0  ...   0.000000   0.000000  0.000000
3                  Brighton   0   0  ...   0.000000   0.000000  0.000000
4                   Everton   0   0  ...   0.000000   0.000000  0.000000
5          Sheffield United   0   0  ...   0.000000   0.000000  0.000000
6          Newcastle United   0   0  ...   0.000000   0.000000  0.000000
7                 Brentford   0   0  ...   0.000000   0.000000  0.000000
8                   Chelsea   0   0  ...   0.000000   0.000000  0.000000
9         Manchester United   0   0  ...   0.000000   0.000000  0.000000
10        Nottingham Forest   1   1  ..

0